<a href="https://colab.research.google.com/github/jh941213/dacon_DoBaeClassification/blob/main/Image_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1nUHbCl6eNwT0ShDueJ5fEpk5--LnnTMo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1nUHbCl6eNwT0ShDueJ5fEpk5--LnnTMo
From (redirected): https://drive.google.com/uc?id=1nUHbCl6eNwT0ShDueJ5fEpk5--LnnTMo&confirm=t&uuid=e64bf5ea-1c0b-4528-be43-7ad9ff533778
To: /content/wallpaper.zip
100% 1.66G/1.66G [00:12<00:00, 129MB/s]


In [2]:
!unzip wallpaper.zip

Archive:  wallpaper.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test/000.png            
  inflating: test/001.png            
  inflating: test/002.png            
  inflating: test/003.png            
  inflating: test/004.png            
  inflating: test/005.png            
  inflating: test/006.png            
  inflating: test/007.png            
  inflating: test/008.png            
  inflating: test/009.png            
  inflating: test/010.png            
  inflating: test/011.png            
  inflating: test/012.png            
  inflating: test/013.png            
  inflating: test/014.png            
  inflating: test/015.png            
  inflating: test/016.png            
  inflating: test/017.png            
  inflating: test/018.png            
  inflating: test/019.png            
  inflating: test/020.png            
  inflating: test/021.png            
  inflating: test/022.png            
  inflating: test/023.png 

In [2]:
import os
import random
import numpy as np
import albumentations as A
from PIL import Image
from tqdm import tqdm


# Albumentations 변환 설정
transform = A.Compose([
            A.Blur(p=0.5),
            # A.RandomResizedCrop(height=224, width=224, scale=(0.5, 1.0), p=0.5),
            A.CoarseDropout(always_apply=False, p=0.5, max_holes=8, max_height=8, max_width=8, min_holes=8, min_height=8, min_width=8, fill_value=(0, 0, 0), mask_fill_value=None),
            A.ElasticTransform(always_apply=False, p=0.5, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, approximate=False, same_dxdy=False),
            A.GaussNoise(always_apply=False, p=0.5, var_limit=(10.0, 50.0), per_channel=True, mean=0.0),
            A.RandomFog(always_apply=False, p=0.5, fog_coef_lower=0.1, fog_coef_upper=0.2, alpha_coef=0.08),
            A.RandomBrightnessContrast(always_apply=False, p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
            A.RandomGamma(always_apply=False, p=0.5, gamma_limit=(80, 120), eps=None),
            A.HorizontalFlip(always_apply=False, p=0.5)
        ])

folders = ["00_Furniture", "04_Corrosion", "05_Loose", "06_Seam", "08_Stain", 
           "09_Gypsum", "12_Crying", "13_Connection", "14_Window&Door", "16_Gap", "17_Piece"]

for folder in folders:
    image_names = os.listdir(folder)
    save_folder_path = folder
    print("진행 폴더 :", folder, "기존 이미지 개수 : ",len(os.listdir(folder)))
    for i in tqdm(range(500)):
        image_name = random.choice(image_names)
        image_path = os.path.join(folder, image_name)

        # 이미지 불러오기
        image = Image.open(image_path)

        # Albumentations 적용
        transformed = transform(image=np.array(image))

        # 적용된 이미지 저장
        save_path = os.path.join(save_folder_path, f"aug_{str(i).zfill(3)}.png")
        Image.fromarray(transformed["image"]).save(save_path)
    print("진행 폴더 :", folder, "증강 이미지 개수 : ", len(os.listdir(folder)))
    print("-----------------------------------------")

FileNotFoundError: ignored

In [ ]:
cd ..

/content


In [6]:
import shutil
shutil.move('/content/train','/content/drive/MyDrive/aug_train')

'/content/drive/MyDrive/aug_train/train'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':123
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [4]:

!cd /content/drive

In [5]:
all_img_list = glob.glob('/content/drive/MyDrive/aug_train/train/*/*')

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[6])

train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [6]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [7]:
train

,img_path,label
1569,/content/drive/MyDrive/aug_train/train/15_Crac...,15
6229,/content/drive/MyDrive/aug_train/train/09_Gyps...,9
990,/content/drive/MyDrive/aug_train/train/10_Cont...,10
4740,/content/drive/MyDrive/aug_train/train/06_Seam...,6
5412,/content/drive/MyDrive/aug_train/train/08_Stai...,8
...,...,...
5623,/content/drive/MyDrive/aug_train/train/08_Stai...,8
4571,/content/drive/MyDrive/aug_train/train/05_Loos...,5
1437,/content/drive/MyDrive/aug_train/train/11_Typo...,11
5737,/content/drive/MyDrive/aug_train/train/08_Stai...,8


In [8]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.feature = []

        for img_path in tqdm(self.img_path_list):
            image = cv2.imread(img_path)
            if self.transforms is not None:
                image = self.transforms(image=image)['image']
            self.feature.append(image)

    def __getitem__(self, index):
        if self.label_list is not None:
            return self.feature[index], self.label_list[index]
        else:
            return self.feature[index]
        
    def __len__(self):
        return len(self.img_path_list)

In [9]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [10]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':128,
    'SEED':123
}

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

  0%|          | 0/6269 [00:00<?, ?it/s]

In [12]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.4 MB/s eta 0:00:00


In [13]:
import timm

In [18]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('tf_efficientnet_b3_ns', pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [19]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [20]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [21]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/196 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
label_map = {0: '가구수정',
             1: '걸레받이수정',
             2: '곰팡이',
             3: '꼬임',
             4: '녹오염',
             5: '들뜸',
             6: '면불량',
             7: '몰딩수정',
             8: '반점',
             9: '석고수정',
             10: '오염',
             11: '오타공',
             12: '울음',
             13: '이음부불량',
             14: '창틀,문틀수정',
             15: '터짐',
             16: '틈새과다',
             17: '피스',
             18: '훼손'}

korean_preds = [label_map[int(pred.split("_")[0])] for pred in preds]

In [ ]:
korean_preds

['훼손',
 '오염',
 '훼손',
 '몰딩수정',
 '오염',
 '훼손',
 '훼손',
 '훼손',
 '오염',
 '오타공',
 '몰딩수정',
 '오타공',
 '오염',
 '오염',
 '훼손',
 '오염',
 '피스',
 '걸레받이수정',
 '오염',
 '오염',
 '가구수정',
 '훼손',
 '훼손',
 '훼손',
 '가구수정',
 '훼손',
 '면불량',
 '오염',
 '훼손',
 '훼손',
 '오염',
 '훼손',
 '훼손',
 '오염',
 '곰팡이',
 '훼손',
 '오염',
 '훼손',
 '오타공',
 '훼손',
 '오염',
 '들뜸',
 '훼손',
 '터짐',
 '오염',
 '훼손',
 '오염',
 '훼손',
 '오염',
 '오염',
 '오염',
 '오염',
 '오염',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '오염',
 '오타공',
 '몰딩수정',
 '훼손',
 '오염',
 '훼손',
 '훼손',
 '오염',
 '오염',
 '오염',
 '터짐',
 '오염',
 '걸레받이수정',
 '오염',
 '면불량',
 '훼손',
 '훼손',
 '훼손',
 '오염',
 '훼손',
 '오염',
 '꼬임',
 '석고수정',
 '오염',
 '훼손',
 '걸레받이수정',
 '훼손',
 '훼손',
 '훼손',
 '꼬임',
 '오염',
 '오염',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '오타공',
 '오염',
 '훼손',
 '훼손',
 '훼손',
 '면불량',
 '훼손',
 '오염',
 '가구수정',
 '몰딩수정',
 '훼손',
 '훼손',
 '면불량',
 '훼손',
 '훼손',
 '훼손',
 '오염',
 '훼손',
 '오염',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '오염',
 '훼손',
 '터짐',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '훼손',
 '오염',
 '꼬임',
 '오염',
 '훼손',
 '오염',
 '훼손',
 '훼

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['label'] = korean_preds

In [ ]:
submit.to_csv('./adfnskdf.csv', index=False, encoding='utf-8-sig')